In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


---

# Objectives

* Identify **high-cost, low-value customers** for re-negotiation or phasing out.
* Surface **strategic high-value accounts** for growth investments.
* Link **financial metrics** (margin, revenue) to **operational impact** (support hours, change orders).
* Enable executive dashboards showing **customer segmentation by value tier**.
* Provide account managers and finance with **actionable insights** for renewal and pricing strategy.

---

# In-Scope Data Sources

* Salesforce
* Workday
* Costpoint
* HCSS
* Viewpoint

---

# Functional Requirements

## Data Ingestion & Integration

* Extract data from:
    * Costpoint
    * Workday
    * Salesforce
    * HCSS
    * Viewpoint
* Standardize and link customer records across systems using **Master Data Management (MDM)** logic.

## Metric Calculation

* **Revenue** from Costpoint/Salesforce.
* **Margin** from Costpoint.
* **Labor effort** (hours \* loaded cost) from HCSS, Viewpoint.
* **Strategic tiering and relationship health** from Salesforce.
* **Customer Cost-to-Value Score** = (Margin % + Tier Score) / (Labor Cost + Support Hours + Discounts)
    * *Additional Information for Calculations: See Customer Value Scoring Framework documentation.*

## Security & Governance

* Mask **employee PII** from Workday for non-HR roles.
* Restrict access to **margin data** to finance users.
* Use **domain-specific roles**.
* Track **source-to-report lineage**.

## Data Cataloging & Metadata

* Catalog each data with tags: **“Customer”**, **“Financial”**, **“Confidential”**.
* Assign **data stewards** for each domain.

## Dashboard & Analytics

* **Executive Cost-to-Value Heatmap**: customers grouped by value quadrant.
* **Top 10 High-Cost Customers**.
* **Customer Lifetime Margin Trend**.
* **Support Cost per \$1M Revenue**.
* **Strategic Account Watchlist** (based on declining value score).

### Loading this data is not much different from anything we have done in previous use case, but this is all json.  Perfectly suited to Snowflake.

In [ ]:
use database cust_cost_value;
use schema raw;
show tables;

### Use the Database browser at left to look at the data in the single Variant column.  We need to flatten the data.  This is done through SQL statements.

I prepared these here: https://app.snowflake.com/us-east-1/zxc20771/wVzUmLdWdAO#query

In [ ]:
use database cust_cost_value;
use schema core;
--Metric Calculation
    --Revenue from Costpoint/Salesforce
    --Margin from Costpoint
    --Labor effort (hours * loaded cost) from HCSS, Viewpoint
    --Strategic tiering and relationship health from Salesforce
    --Final: Customer Cost-to-Value Score = (Margin % + Tier Score) / (Labor Cost + Support Hours +Discounts)
--Additional Information for Calculations: See Customer Value Scoring Framework documentation.

select distinct metricname from CUST_COST_VALUE.CORE.CUSTOMER_VALUE_METRICS_VIEW;
select distinct metricname from CUST_COST_VALUE.CORE.COSTPOINT_CLIENT_VALUE_METRICS_VIEW;
select distinct metricname from CUST_COST_VALUE.CORE.SALESFORCE_CUSTOMER_VALUE_METRICS_VIEW;




In [ ]:
select accountid, b.name, metricname, metricvalue from CUST_COST_VALUE.CORE.SALESFORCE_CUSTOMER_VALUE_METRICS_VIEW a, salesforce_accounts_view b
where metricname = 'NPS' and a.accountid= b.id;
--hmm there history in the metrics table, need to get the most current

## We're going to use CoPilot to help us create a few queries.  This is an incredible tool right within Snowflake and it WILL speed your development whether you are a data engineer or an analyst.

Try: The previous query needs to be modified to retrieve the latest metricvalue per accountid using the asofdate field

In [ ]:
WITH LatestNPSMetrics AS (
  SELECT
    accountid,
    metricname,
    metricvalue,
    asofdate,
    ROW_NUMBER() OVER (
      PARTITION BY accountid
      ORDER BY
        asofdate DESC
    ) AS rn
  FROM
    CUST_COST_VALUE.CORE.SALESFORCE_CUSTOMER_VALUE_METRICS_VIEW
  WHERE
    metricname ILIKE '%NPS%'
)
SELECT
  a.ACCOUNTID,
  b.NAME,
  a.METRICNAME,
  a.METRICVALUE AS ORIGINAL_VALUE,
  ROUND((a.METRICVALUE + 100) / 20, 1) AS METRICVALUE_10PT_SCALE
FROM
  LatestNPSMetrics AS a
  JOIN CUST_COST_VALUE.CORE.SALESFORCE_ACCOUNTS_VIEW AS b ON a.ACCOUNTID = b.ID
WHERE
  rn = 1;

In [ ]:
WITH LatestNPSMetrics AS (
  SELECT
    accountid,
    metricname,
    metricvalue,
    ROUND((metricvalue + 100) / 20, 1) AS NORMALIZED_VALUE,
    asofdate,
    ROW_NUMBER() OVER (
      PARTITION BY accountid
      ORDER BY
        asofdate DESC
    ) AS rn
  FROM
    CUST_COST_VALUE.CORE.SALESFORCE_CUSTOMER_VALUE_METRICS_VIEW
  WHERE
    metricname ILIKE '%NPS%'
)
SELECT
  a.ACCOUNTID,
  b.NAME,
  a.METRICNAME,
  a.METRICVALUE AS ORIGINAL_VALUE,
  a.NORMALIZED_VALUE AS METRICVALUE_10PT_SCALE,
  CASE
    WHEN a.NORMALIZED_VALUE BETWEEN 9
    AND 10 THEN 20
    WHEN a.NORMALIZED_VALUE BETWEEN 7
    AND 8 THEN 10
    ELSE 0
  END AS NPS_SCORE
FROM
  LatestNPSMetrics AS a
  JOIN CUST_COST_VALUE.CORE.SALESFORCE_ACCOUNTS_VIEW AS b ON a.ACCOUNTID = b.ID
WHERE
  rn = 1;

In [ ]:

--Revenue Tier from Salesforce    
SELECT
    ID,
    NAME,
    AMOUNT,
    Stage,
    closedate,
    CASE
        WHEN AMOUNT > 5000000 THEN 15
        WHEN AMOUNT >= 1000000 AND AMOUNT <= 5000000 THEN 10
        WHEN AMOUNT < 1000000 THEN 5
        ELSE 0 -- Handle cases where AMOUNT might be NULL or unexpected
    END AS REVENUE_TIER_POINTS
FROM
    CUST_COST_VALUE.CORE.SALESFOCE_OPPORTUNITIES_VIEW
WHERE
    STAGE = 'Closed Won'
QUALIFY ROW_NUMBER() OVER (PARTITION BY ID ORDER BY closedate DESC) = 1;





In [ ]:
select * from 
CUST_COST_VALUE.CORE.VIEWPOINT_CLIENTS a,
CUST_COST_VALUE.CORE.SALESFORCE_ACCOUNTS_VIEW b,
CUST_COST_VALUE.CORE.HCSS_CUSTOMERS_VIEW c
where a.clientname = b.name and b.name = c.customername;